## Test Open Weather api key

In [6]:
import os
import time
from dotenv import load_dotenv
from julep import Julep

In [7]:
load_dotenv()

True

In [8]:
client = Julep(
    api_key=os.getenv('JULEP_API_KEY'),
    environment=os.getenv('JULEP_ENVIRONMENT', 'production')
)

In [9]:
agent = client.agents.create(
    name="Weather Reporter",
    model="claude-3.5-sonnet",
    about="An AI agent that tells about the weather condidion of the follwing place."
)

In [12]:
task_definition = {
    "name": "Foodie Tour Generator",
    "description": "Generates a one-day foodie tour for each city based on weather and local dishes.",
    "input_schema": {
        "type": "object",
        "properties": {
            "cities": {
                "type": "array",
                "items": {"type": "string"}
            }
        }
    },
    "tools": [
        {
            "name": "weather",
            "type": "integration",
            "integration": {
                "provider": "weather",
                "setup": {
                    "openweathermap_api_key": os.getenv('OPENWEATHERMAP_API_KEY')
                }
            }
        }
    ],
    "main": [
        {
            "over": "$ steps[0].input.cities",
            "map": {
                "tool": "weather",
                "arguments": {
                    "location": "$ _"
                }
            }
        }
    ]
}

In [13]:
task = client.tasks.create(
    agent_id=agent.id,
    **task_definition
)

In [14]:
execution = client.executions.create(
    task_id=task.id,
    input={"cities": ["Paris", "Tokyo", "New York"]}
)

In [16]:
print(execution)

Execution(id='0683feb2-3eaf-708a-8000-4edea35b1a0a', created_at=datetime.datetime(2025, 6, 4, 6, 43, 47, 920659, tzinfo=datetime.timezone.utc), input={'cities': ['Paris', 'Tokyo', 'New York']}, status='queued', task_id='0683feb1-9be6-78ac-8000-900b88f7c881', updated_at=datetime.datetime(2025, 6, 4, 6, 43, 47, 958490, tzinfo=datetime.timezone.utc), error=None, metadata={'jobs': ['0683feb2-4093-78af-8000-d0f1717fc70b']}, output=None, transition_count=None)


In [17]:
while (result := client.executions.get(execution.id)).status not in ['succeeded', 'failed']:
    print(result.status)
    time.sleep(1)

In [18]:
if result.status == "succeeded":
    print(result.output)
else:
    print(f"Error: {result.error}")

[{'result': 'In Paris, the current weather is as follows:\nDetailed status: overcast clouds\nWind speed: 4.12 m/s, direction: 200°\nHumidity: 82%\nTemperature: \n  - Current: 14.32°C\n  - High: 15.43°C\n  - Low: 13.54°C\n  - Feels like: 13.95°C\nRain: {}\nHeat index: None\nCloud cover: 100%'}, {'result': 'In Tokyo, the current weather is as follows:\nDetailed status: broken clouds\nWind speed: 5.66 m/s, direction: 180°\nHumidity: 61%\nTemperature: \n  - Current: 27.24°C\n  - High: 28.13°C\n  - Low: 25.75°C\n  - Feels like: 28.47°C\nRain: {}\nHeat index: None\nCloud cover: 75%'}, {'result': 'In New York, the current weather is as follows:\nDetailed status: clear sky\nWind speed: 2.06 m/s, direction: 270°\nHumidity: 63%\nTemperature: \n  - Current: 17.34°C\n  - High: 18.52°C\n  - Low: 14.9°C\n  - Feels like: 16.77°C\nRain: {}\nHeat index: None\nCloud cover: 0%'}]
